In [1]:
'''
Title: Batter Performance Table Pt 2
Author: Sam LaFell
Date: 11/5/2022
Purpose: Part 1 didnt seem to work

Granularity: xBA by pitch for each batter. By pitcher handiness.
'''

# Import Libraries
import pandas as pd
pd.set_option('display.max_columns', 500)
import pybaseball as pb
import os
import numpy as np
import re
from datetime import datetime, timedelta

In [2]:
import sys
import pathlib
import os 

# Set base path
base_path = os.path.abspath('/Users/samlafell/Desktop/sports_betting_project')
os.chdir(base_path)

# Add utils folder if not already in system path
if os.path.join(base_path, 'utils') not in sys.path:
    print('adding path')
    sys.path.append(os.path.join(base_path, 'utils'))
    
from utils import get_players, game_performances, plate_appearances, expected_ba

adding path


## Lookup the Player IDs for all players on a specific Team
## Then use this to get their performance information for games in a time period leading up to a certain date

In [3]:
# Generate a lookup table to use in the Game Performances function
atl_players = get_players.get_players('2018-05-13', 'ATL', min_abs_season=10)

Gathering player lookup table. This may take a moment.


In [5]:
# If Game Date is after 5/1, do this, else we will need previous season numbers
game_performances_df = game_performances.game_performances('2018-05-13', atl_players, lookback_period=28)

Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data


In [6]:
# Get plate appaearnce info in this time
atbats = plate_appearances.plate_appearances(game_performances_df, min_pas = 15, p_handiness=False)

# Get Expected BA Count
expected_ba_df = expected_ba.expected_ba(game_performances_df, atbats.batter.values, p_handiness=False)

# Batters PAs and xBA to date
pas_xba = atbats.merge(expected_ba_df, how='inner', on='batter').rename(columns={'estimated_ba_using_speedangle':'xBA'})
pas_xba.sort_values('xBA', ascending=False)

,batter,PA,xBA
5,518692,106,0.420241
2,455976,108,0.380409
1,452095,25,0.370882
4,518586,22,0.361462
7,621020,62,0.345474
10,660670,72,0.344102
6,542255,110,0.335621
9,645277,113,0.307402
3,475247,62,0.303538
8,622666,56,0.301314


In [8]:
# Get plate appaearnce info in this time
atbats = plate_appearances.plate_appearances(game_performances_df, min_pas = 15, p_handiness=True)

# Get Expected BA Count
expected_ba_df = expected_ba.expected_ba(game_performances_df, atbats.batter.values, p_handiness=True)

# Batters PAs and xBA to date
pas_xba = atbats.merge(expected_ba_df, how='inner', on='batter').rename(columns={'estimated_ba_using_speedangle':'xBA'})
pas_xba.sort_values('xBA', ascending=False)

NameError: name 'handiness' is not defined

In [8]:
# Get Expected BA Count
expected_ba_df = expected_ba.expected_ba_phandiness(game_performances_df, atbats.batter.values)
# Batters PAs and xBA to date
pas_xba = atbats.merge(expected_ba_df, how='inner', on='batter').rename(columns={'estimated_ba_using_speedangle':'xBA'})
pas_xba.sort_values('xBA', ascending=False)

,batter,PA,p_throws,xBA
14,621020,62,L,0.498600
2,452095,25,L,0.445200
11,518692,106,R,0.430434
10,518692,106,L,0.399462
0,435559,72,L,0.389800
4,455976,108,L,0.387879
5,455976,108,R,0.376300
20,660670,72,L,0.374650
8,518586,22,L,0.374000
12,542255,110,L,0.367000


In [29]:
# The batters before this game
season_estimated_bapt1 = atl_batting_performances.groupby(['batter', 'p_throws']).agg({'estimated_ba_using_speedangle':'mean'}).reset_index()

season_expected_ba = pd.pivot(season_estimated_ba, index='batter', columns='p_throws', values='estimated_ba_using_speedangle').reset_index()
season_expected_ba.columns = ['batter', 'lefty_pitcher', 'righty_pitcher']

In [63]:
# PAs before this game
atbats = atl_batting_performances.groupby(['game_date', 'batter', 'at_bat_number']).agg({'batter':'count'})
atbats.columns = ['batter_count']
atbats = atbats.reset_index()
season_atbats = atbats.drop(columns=['at_bat_number'], axis=1).groupby('batter').sum(numeric_only=True).reset_index()

# Batters PAs and xBA to date
pas_xba = season_atbats.merge(season_expected_ba, how='inner', on='batter')
pas_xba.columns = ['batter', 'PA', 'lefty_xBA', 'righty_xBA']

# Average xBA. Essentially coalescing if one is NA, use the one that's not NA.
pas_xba['Avg_xBA'] = np.where((((pas_xba['lefty_xBA'] + pas_xba['lefty_xBA']) / 2).isna()) == False, \
                                    ((pas_xba['lefty_xBA'] + pas_xba['righty_xBA']) / 2), \
                                    np.where(pas_xba['lefty_xBA'].isna(), 
                                             pas_xba['righty_xBA'],
                                             pas_xba['lefty_xBA']))


,batter,PA,lefty_xBA,righty_xBA,Avg_xBA
0,435221,30,0.053000,0.087000,0.070000
1,435559,157,0.358143,0.230172,0.294158
2,452095,91,0.445200,0.339917,0.392558
3,455976,320,0.395767,0.411976,0.403871
4,475247,155,0.335667,0.328727,0.332197
5,488721,1,0.313000,NaN,0.000000
6,518586,58,0.374000,0.353333,0.363667
7,518692,287,0.381375,0.449675,0.415525
8,527054,17,0.583500,0.250000,0.416750
9,542255,313,0.371947,0.299689,0.335818


In [65]:
# Minimum PA to qualify
min_qual = 25
pas_xba.loc[pas_xba['PA'] >= min_qual]

,batter,PA,lefty_xBA,righty_xBA,Avg_xBA
0,435221,30,0.053000,0.087000,0.070000
1,435559,157,0.358143,0.230172,0.294158
2,452095,91,0.445200,0.339917,0.392558
3,455976,320,0.395767,0.411976,0.403871
4,475247,155,0.335667,0.328727,0.332197
6,518586,58,0.374000,0.353333,0.363667
7,518692,287,0.381375,0.449675,0.415525
9,542255,313,0.371947,0.299689,0.335818
10,592314,27,NaN,0.163500,0.163500
12,621020,147,0.498600,0.294500,0.396550
